# The effect of using hyper parameter hints

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import seaborn as sns # prettify matplotlib

import numpy as np
import sklearn.gaussian_process as gp

In [ ]:
# local modules
import turbo as tb
import turbo.modules as tm
import turbo.plotting as tp
import turbo.gui as tg

Function to optimize:

In [ ]:
f = lambda x: 1 * x * np.cos(x)
xmin, xmax = 0, 12
xs = np.linspace(xmin, xmax, num=200)

ys = f(xs)
best_y = np.min(ys)
best_x = xs[np.argmin(ys)]

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(xs, ys, 'g-', label='objective')
plt.plot(best_x, best_y, 'bo', label='optima')
plt.legend(loc='upper left')
plt.margins(0.01, 0.1)
plt.xlabel(r'$x$')
plt.ylabel(r'$f(x)$')
plt.show()

In [ ]:
def create_optimiser(use_hint):
    bounds = [
        ('x', xmin, xmax)
    ]

    op = tb.Optimiser(f, 'min', bounds)
    op.latent_space = tm.NoLatentSpace()
    op.plan = tm.Plan(pre_phase_trials=3)
    op.pre_phase_select = tm.random_selector()
    op.maximise_acq = tm.random_quasi_newton(num_random=100, grad_restarts=5)
    op.async_eval = None
    op.surrogate_factory = tm.SciKitGPSurrogate.Factory(gp_params=dict(
        alpha = 1e-3, # larger => more noise. Default = 1e-10
        kernel = 1.0 * gp.kernels.RBF(),
        n_restarts_optimizer = 10,
        normalize_y = True,
        copy_X_train = True # make a copy of the training data
    ), use_hint=use_hint)
    op.acq_func_factory = tm.UCB.Factory(beta=3)
    return op

def compare(a, b, action, vertical):
    rows, cols = (2, 1) if vertical else (1, 2)
    size= (12, 5) if vertical else (12, 5)
    fig, (a1, a2) = plt.subplots(rows, cols, figsize=size)
    action(a, (fig, a1))
    action(b, (fig, a2))
    a1.set_title(a1.get_title() + ' - without hint')
    a2.set_title(a2.get_title() + ' - with hint')
    fig.subplots_adjust(hspace=0.5)

In [ ]:
a = create_optimiser(False)
ra = tp.PlottingRecorder()
a.register_listener(ra)

b = create_optimiser(True)
rb = tp.PlottingRecorder()
b.register_listener(rb)

In [ ]:
np.random.seed(100)
tg.OptimiserProgressBar(a)
a.run(max_trials=20)

np.random.seed(100)
tg.OptimiserProgressBar(b)
b.run(max_trials=20)

In [ ]:
compare(a, b, lambda op, fig_ax:
    tp.plot_error_over_time(op, true_best=best_y, fig_ax=fig_ax), vertical=True)

In [ ]:
compare(ra, rb, lambda rec, fig_ax:
    tp.plot_surrogate_likelihood_over_time(rec, fig_ax=fig_ax), vertical=True)

In [ ]:
compare(ra, rb, lambda rec, fig_ax:
    tp.plot_surrogate_hyper_params_1D(rec, param_index=1, axes=('trial_num', 'param', 'likelihood'), fig_ax=fig_ax), vertical=False)

In [ ]:
compare(ra, rb, lambda rec, fig_ax:
    tp.plot_surrogate_hyper_params_1D(rec, param_index=1, axes=('param', 'likelihood', 'trial_num'), fig_ax=fig_ax), vertical=False)

In [ ]:
compare(ra, rb, lambda rec, fig_ax:
    tp.plot_surrogate_hyper_params_1D(rec, param_index=1, axes=('trial_num', 'likelihood', 'param'), fig_ax=fig_ax), vertical=False)

In [ ]:
compare(ra, rb, lambda rec, fig_ax:
    tp.plot_surrogate_hyper_params_2D(rec, fig_ax=fig_ax), vertical=False)

In [ ]:
tp.plot_trial_1D(rb, param='x', trial_num=None, true_objective=f)